__Diet Problem__

https://web.mit.edu/15.053/www/Excel_Solver.pdf

In [1]:
import pandas as pd

from pulp import LpProblem, LpStatus, LpVariable, LpAffineExpression
from pulp import LpBinary, LpMinimize, LpMaximize, lpSum, value

In [2]:
df = pd.DataFrame(
    [
        ["Hummus", 6, 0.1, 0.05, 0.033],
        ["Moussaka", 10, 0.2, 0.067, 0.067],
        ["Tabouleh", 4.5, 0.067, 0.04, 0.033],
    ],
    columns=["food", "profit", "cooking", "packaging", "delivery"],
)

df

,food,profit,cooking,packaging,delivery
0,Hummus,6.0,0.100,0.050,0.033
1,Moussaka,10.0,0.200,0.067,0.067
2,Tabouleh,4.5,0.067,0.040,0.033


In [3]:
cooking = dict(zip(df.food, df.cooking))
packaging = dict(zip(df.food, df.packaging))
delivery = dict(zip(df.food, df.delivery))

profit = dict(zip(df.food, df.profit))

food_vars = LpVariable.dict("food_vars", df.food.values, lowBound=0, cat="Integer")

In [9]:
model = LpProblem("Food Start-Up", LpMaximize)

# Objective function
model += lpSum([profit[food] * food_vars[food] for food in df.food])

# Constraints
model += (
    lpSum([cooking[food] * food_vars[food] for food in df.food]) <= 4,
    "Total cooking time",
)
model += (
    lpSum([packaging[food] * food_vars[food] for food in df.food]) <= 2,
    "Total packaging time",
)
model += (
    lpSum([delivery[food] * food_vars[food] for food in df.food]) <= 2,
    "Total delivery time",
)

# We can only make 30 tabouleh orders
model += food_vars["Tabouleh"] <= 30

In [10]:
results = model.solve()

LpStatus[model.status]

'Optimal'

In [11]:
output = df[["food", "profit"]].copy()
output["orders"] = [food_vars[food].value() for food in df.food]

output

,food,profit,orders
0,Hummus,6.0,17.0
1,Moussaka,10.0,4.0
2,Tabouleh,4.5,22.0


In [12]:
(output.profit * output.orders).sum()

241.0